## Homework: Evaluation and Monitoring

In this homework, we'll evaluate the quality of our RAG system.

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Getting the data

Let's start by getting the dataset. We will use the data we generated in the module.

In particular, we'll evaluate the quality of our RAG system
with [gpt-4o-mini](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv)


Read it:

```python
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
```

We will use only the first 300 documents:


```python
df = df.iloc[:300]
```

In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rouge import Rouge

/home/ubuntu/miniconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

## Q1. Getting the embeddings model

Now, get the embeddings model `multi-qa-mpnet-base-dot-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

> Note: this is not the same model as in HW3

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embeddings for the first LLM answer:

```python
answer_llm = df.iloc[0].answer_llm
```

What's the first value of the resulting vector?

* -0.42
* -0.22
* -0.02
* 0.21

In [4]:
model_name = 'multi-qa-mpnet-base-dot-v1'
model = SentenceTransformer(model_name)

In [5]:
answer_llm = df.iloc[0].answer_llm

In [6]:
model.encode(answer_llm)[0]

-0.4224469

## Q2. Computing the dot product


Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the `evaluations` list

What's the 75% percentile of the score?

* 21.67
* 31.67
* 41.67
* 51.67

In [7]:
evaluations = []

for _, row in df.iterrows():
    llm_embedding = model.encode(row['answer_llm'])
    orig_embedding = model.encode(row['answer_orig'])
    dot_product = np.dot(llm_embedding, orig_embedding)
    evaluations.append(dot_product)

In [8]:
evaluations[:5]

[17.515991, 13.418406, 25.313255, 12.147419, 18.747728]

In [9]:
percentile_75 = np.percentile(evaluations, 75)

In [10]:
percentile_75

31.674306869506836

## Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we 

* Compute the norm of a vector
* Divide each element by this norm

So, for vector `v`, it'll be `v / ||v||`

In numpy, this is how you do it:

```python
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Let's put it into a function and then compute dot product 
between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

* 0.63
* 0.73
* 0.83
* 0.93

In [11]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [12]:
evaluations = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    llm_embedding = model.encode(row['answer_llm'])
    orig_embedding = model.encode(row['answer_orig'])

    llm_embedding_norm = normalize(llm_embedding)
    orig_embedding_norm = normalize(orig_embedding)

    cosine_similarity = np.dot(llm_embedding_norm, orig_embedding_norm)
    evaluations.append(cosine_similarity)

100%|██████████| 300/300 [02:05<00:00,  2.40it/s]


In [13]:
percentile_75 = np.percentile(evaluations, 75)
print(f"75th percentile of cosine similarity scores: {percentile_75:.3f}")

75th percentile of cosine similarity scores: 0.836


In [14]:
percentile_75

0.8362348675727844

## Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.  

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```

(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

What's the F score for `rouge-1`?

- 0.35
- 0.45
- 0.55
- 0.65

In [15]:
rouge_scorer = Rouge()

r = df.iloc[10]

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

rouge_1_f1 = scores['rouge-1']['f']

print(f"ROUGE-1 F1 score: {rouge_1_f1:.2f}")

ROUGE-1 F1 score: 0.45


In [16]:
r

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
Name: 10, dtype: object

In [17]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [18]:
rouge_1_f1

0.45454544954545456

## Q5. Average rouge score

Let's compute the average F-score between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4

- 0.35
- 0.45
- 0.55
- 0.65

In [19]:
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']

average_f_score = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3

In [20]:
average_f_score

0.35490034990035496

## Q6. Average rouge score for all the data points

Now let's compute the score for all the records and create a dataframe from them.

What's the average `rouge_2` across all the records?

- 0.10
- 0.20
- 0.30
- 0.40

In [21]:
rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Computing ROUGE scores"):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    
    rouge_1_scores.append(scores['rouge-1']['f'])
    rouge_2_scores.append(scores['rouge-2']['f'])
    rouge_l_scores.append(scores['rouge-l']['f'])

scores_df = pd.DataFrame({
    'rouge_1': rouge_1_scores,
    'rouge_2': rouge_2_scores,
    'rouge_l': rouge_l_scores
})

average_rouge_2 = scores_df['rouge_2'].mean()

Computing ROUGE scores: 100%|██████████| 300/300 [00:01<00:00, 253.95it/s]


In [22]:
average_rouge_2

0.20696501983423318